<a href="https://colab.research.google.com/github/stbhg5/ComputerScience/blob/main/python_ch13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  파이썬의 속도 문제에 대해

In [ ]:
# 일반 파이썬 for문 계산
import time
start = time.time()
count = 0
for i in range(10000):
    for j in range(10000):
        count += 1
print(time.time() - start)
# time.sleep(1000)

12.0969877243042


In [ ]:
# 기본 파이썬 인터프리터가 아닌 pypy3 JIT Complier 사용
# https://www.pypy.org/download.html

In [ ]:
# scikit-learn 라이브러리
# https://scikit-learn.org/stable/developers/performance.html
# 위 참고한 파이썬 속도 빠르게 하는 법
# 1. numpy, scipy 라이브러리 사용
# 2. 잘 작성된 BSD or MIT 라이센스 C/C++ 라이브러리 사용하여 Cython wrapper를 사용하여 쓰기
# 3. Cython 다이렉트로 쓰기
# 4. 멀티 프로세싱 쓰기

# scipy github
# https://github.com/scipy/scipy
# scipy 라이센스 : BSD (BSD나 MIT는 라이센스가 약간 자유로운 편)
# https://github.com/scipy/scipy/blob/main/LICENSE.txt
# (GPL LGPL 라이센스는 좀 까다로운 편)

# 등등 라이브러리 여러가지 쓰기

In [ ]:
# cython 사용
!python setup.py build_ext --inplace

Compiling hello.pyx because it changed.
[1/1] Cythonizing hello.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/hello.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'hello' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c hello.c -o build/temp.linux-x86_64-3.8/hello.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format

In [1]:
# google colab에서 cython 사용
%load_ext Cython

In [33]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

# 느린 모듈(병목이 생기는 모듈)
# class check_odd_number:
#     def check(self, number):
#         if number % 2 == 1:
#             return True
#         else:
#             return False

# python_class = check_odd_number()

# 느린 모듈을 사이썬 정의로 변경
cpdef check(int number):
    if number % 2 == 1:
        return True
    else:
        return False

# 타입 선언
# cdef는 c로된 코드 쓰겠다는 것이고 cpdef는 c와 파이썬으로 된 코드 같이 쓰겠다.
cpdef cython_add(int num1, int num2):
    cdef int result, i, j
    result = 0
    for i in range(num1):
        for j in range(num2):
            # if python_class.check(i):
            if check(i):
                continue
            result += i * j
            # result += 1
    return result

In [34]:
# 일반 파이썬 함수
def python_add(num1, num2):
    result = 0
    for i in range(num1):
        for j in range(num2):
            # if python_class.check(i):
            if check(i):
                continue
            result += i * j
            # result += 1
    return result

In [35]:
# 일반 파이썬 함수의 계산
import time
start = time.time()
python_result = python_add(10000, 10000)
print(time.time() - start)

13.299113512039185


In [36]:
# 사이썬 함수의 계산
import time
start = time.time()
cython_result = cython_add(10000, 10000)
print(time.time() - start)

0.42444300651550293


In [19]:
python_result

100000000

In [20]:
# int 한계치로 857419840가 나옴
cython_result

100000000

In [ ]:
# cython에 대한 자세한 내용은 https://cython.org/ 참고
# + 잘 짠 코드 모방하기 - scikit-learn

In [38]:
# cython 사용시 주의점
%%cython
import numpy as np
cimport numpy as np
cimport cython

# cpdef add(int[:] list):
# cpdef add(int[:] array): # MemoryView of 'ndarray' object 오류 뜬다.
cpdef add(long[:] list): # cython의 list
    cdef int result, i
    result = 0
    for i in list:
        result += i
    return result

In [39]:
import numpy as np
# add(np.array([i for i in range(10000)], dtype = np.int32))
# add(np.array([i for i in range(10000)]).astype('int32'))
add(np.array([i for i in range(10000)])) # numpy array 자료형

# c에서의 데이터 타입과 파이썬에서의 데이터 타입이 다르기 때문에 이 부분 주의하기

49995000

In [45]:
# 일반 파이썬 for문으로 api 실행
import time
start = time.time()
result = []
for i in range(10):
    result.append(requests.get(url).json())
print(time.time() - start)

4.708981275558472


In [46]:
# 멀티 쓰레딩이 네트워크 관련 업무 빠르게 할 수 있다.
from threading import Thread
import requests

# 이 url은 여러개가 들어와도 순차적으로 실행하도록 되어 있는 것으로 보임.
url = 'http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?overview=false' # http://project-osrm.org/docs/v5.5.1/api/#general-options 의 API URL 사용

def work(url, index):
    result[index] = requests.get(url).json()

if __name__ == '__main__':
    start = time.time()
    result = [None] * 10
    # threads = [None] * 10 # join() 사용 예시이지만 현재 사용하는 url이 예시에 적절치 못하므로 패스
    for i in range(10):
        th1 = Thread(target = work, args = (url, i))
        # threads[i] = Thread(target = work, args = (url, i))
        th1.start()
        # threads[i].start()
        # for i in range(10):
        #     threads[i].join()
        # th1.join() # join() 메서드는 파이썬에게 프로세스가 종료 될 때까지 대기하도록 지시
    print(time.time() - start)
time.sleep(1) # join으로 인해 일반 파이썬으로 api 실행시 똑같은 시간이 걸리는 것으로 보여서 join을 없앨 시 1초 시간 텀을 주면 멀티 쓰레딩 사용시 api 실행할 때 1초도 안걸리는 것을 알 수 있음
result

0.035839080810546875


[{'code': 'Ok',
  'routes': [{'legs': [{'steps': [],
      'summary': '',
      'weight': 262.8,
      'duration': 260,
      'distance': 1887},
     {'steps': [],
      'summary': '',
      'weight': 370.4,
      'duration': 370.4,
      'distance': 2845.5}],
    'weight_name': 'routability',
    'weight': 633.2,
    'duration': 630.4,
    'distance': 4732.5}],
  'waypoints': [{'hint': 'pv8JgK8GmoUXAAAABQAAAAAAAAAgAAAAIXRPQYXNK0AAAAAAcPePQQsAAAADAAAAAAAAABAAAABH9wAA_kvMAKlYIQM8TMwArVghAwAA7wolVKRv',
    'distance': 4.231521214,
    'name': 'Friedrichstraße',
    'location': [13.388798, 52.517033]},
   {'hint': 'CCzdgZTUiYcGAAAACgAAAAAAAAB3AAAAppONQOodwkAAAAAA8TeEQgYAAAAKAAAAAAAAAHcAAABH9wAAfm7MABiJIQOCbswA_4ghAwAAXwUlVKRv',
    'distance': 2.795148358,
    'name': 'Torstraße',
    'location': [13.39763, 52.529432]},
   {'hint': 'uyoYgP___38fAAAAUQAAACYAAAAeAAAAeosKQlNOX0Jq6iZCjsMGQh8AAABRAAAAJgAAAB4AAABH9wAASufMAOdwIQNL58wA03AhAwQAvxAlVKRv',
    'distance': 2.226580806,
    'name': 'P